In [1]:
import pandas as pd

In [2]:
clean_data = pd.read_csv("../data/processed/01-ks-projects-clean.csv")
feat_data = pd.read_csv("../data/processed/02-ks-projects-engineered.csv")

In [3]:
clean_data.state = clean_data.state.map(dict(successful=1, failed=0))
feat_data.state = feat_data.state.map(dict(successful=1, failed=0))
feat_data

,ID,name,category,main_category,currency,deadline,launched,state,backers,country,...,deadline_year,deadline_month,deadline_day,launched_year,launched_month,launched_day,launched_hour,launched_minute,name_length,name_words
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,0,0,GB,...,2015,October,Friday,2015,August,Tuesday,12,12,31,5
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,2017-09-02 04:43:57,0,15,US,...,2017,November,Wednesday,2017,September,Saturday,4,43,45,8
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,2013-01-12 00:20:50,0,3,US,...,2013,February,Tuesday,2013,January,Saturday,0,20,14,3
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,2012-03-17 03:24:11,0,1,US,...,2012,April,Monday,2012,March,Saturday,3,24,49,7
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,2016-02-26 13:38:27,1,224,US,...,2016,April,Friday,2016,February,Friday,13,38,20,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329416,999975836,"Homemade fresh dog food, Cleveland OH",Small Batch,Food,USD,2017-04-19,2017-03-20 22:08:22,0,4,US,...,2017,April,Wednesday,2017,March,Monday,22,8,37,6
329417,999977640,The Tribe,Narrative Film,Film & Video,USD,2011-07-19,2011-06-22 03:35:14,0,5,US,...,2011,July,Tuesday,2011,June,Wednesday,3,35,9,2
329418,999986353,Walls of Remedy- New lesbian Romantic Comedy f...,Narrative Film,Film & Video,USD,2010-08-16,2010-07-01 19:40:30,0,1,US,...,2010,August,Monday,2010,July,Thursday,19,40,71,11
329419,999987933,BioDefense Education Kit,Technology,Technology,USD,2016-02-13,2016-01-13 18:13:53,0,6,US,...,2016,February,Saturday,2016,January,Wednesday,18,13,24,3


In [4]:
clean_data = clean_data.drop(columns=["backers", "usd_pledged_real", "ID", "launched", "deadline", "name"])
feat_data = feat_data.drop(columns=["backers", "usd_pledged_real", "launched", "deadline", "ID", "name"])

In [5]:
dum_clean_data = pd.get_dummies(clean_data, columns=["category", "main_category", "currency", "country"])
dum_feat_data = pd.get_dummies(feat_data, columns=["category", "main_category", "currency", "country", "deadline_month", "deadline_day", "launched_month", "launched_day"])

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

y_clean_col = clean_data.state
X_clean_train = dum_clean_data.drop(columns=["state"])
X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X_clean_train, y_clean_col, test_size = 0.3, random_state=1)
X_clean_train, X_clean_val, y_clean_train, y_clean_val = train_test_split(X_clean_train, y_clean_train, test_size = 0.3, random_state=1)

y_feat_col = feat_data.state
X_feat_train = dum_clean_data.drop(columns=["state"])
X_feat_train, X_feat_test, y_feat_train, y_feat_test = train_test_split(X_feat_train, y_feat_col, test_size = 0.3, random_state=1)
X_feat_train, X_feat_val, y_feat_train, y_feat_val = train_test_split(X_feat_train, y_feat_train, test_size = 0.3)

In [7]:
model1_1 = KNeighborsClassifier().fit(X_clean_train, y_clean_train)
model1_2 = KNeighborsClassifier().fit(X_feat_train, y_feat_train)
model2_1 = RandomForestClassifier().fit(X_clean_train, y_clean_train)
model2_2 = RandomForestClassifier().fit(X_feat_train, y_feat_train)
model3_1 = DecisionTreeClassifier().fit(X_clean_train, y_clean_train)
model3_2 = DecisionTreeClassifier().fit(X_feat_train, y_feat_train)

In [8]:
from sklearn.metrics import accuracy_score

results = pd.DataFrame(columns=["model", "accuracy"])
acc = accuracy_score(y_clean_val, model1_1.predict(X_clean_val))
results = results.append({"model": "model1_1", "accuracy": acc}, ignore_index=True)
acc = accuracy_score(y_clean_val, model2_1.predict(X_clean_val))
results = results.append({"model": "model2_1", "accuracy": acc}, ignore_index=True)
acc = accuracy_score(y_clean_val, model3_1.predict(X_clean_val))
results = results.append({"model": "model3_1", "accuracy": acc}, ignore_index=True)
acc = accuracy_score(y_feat_val, model1_2.predict(X_feat_val))
results = results.append({"model": "model1_2", "accuracy": acc}, ignore_index=True)
acc = accuracy_score(y_feat_val, model2_2.predict(X_feat_val))
results = results.append({"model": "model2_2", "accuracy": acc}, ignore_index=True)
acc = accuracy_score(y_feat_val, model3_2.predict(X_feat_val))
results = results.append({"model": "model3_2", "accuracy": acc}, ignore_index=True)

In [9]:
results.sort_values(by = 'accuracy', ascending = False)

,model,accuracy
1,model2_1,0.643779
2,model3_1,0.641524
4,model2_2,0.638619
5,model3_2,0.637592
0,model1_1,0.618092
3,model1_2,0.614464


In [14]:
criterion = ["gini", "entropy"]
n_estimators = [90, 100, 110]
min_samples_split = [2, 5, 9, 11]
min_samples_leaf = [1,2,3]
min_weight_fraction_leaf = [0.0, 0.001, 0.002]
max_features = ["auto", "sqrt", "log2"]

results_clean = pd.DataFrame(columns=['criterion', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'min_weight_fraction_leaf', 'max_features', 'accuracy'])
results_feat = pd.DataFrame(columns=['criterion', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'min_weight_fraction_leaf', 'max_features', 'accuracy'])

for i in criterion:
    for j in n_estimators:
        for k in min_samples_split:
            for l in min_samples_leaf:
                for m in min_weight_fraction_leaf:
                    for n in max_features:
                        model = RandomForestClassifier(criterion = i, n_estimators = j, min_samples_split = k, min_samples_leaf = l, min_weight_fraction_leaf = m, max_features = n)
                        model.fit(X_clean_train, y_clean_train)
                        acc = accuracy_score(y_clean_val, model.predict(X_clean_val))
                        results_clean = results_clean.append({'max_depth': j, 'min_samples_split': k, 'min_samples_leaf': l, 'min_weight_fraction_leaf': m, 'max_features': n, 'criterion': i, 'accuracy': acc }, ignore_index=True)
                        model.fit(X_feat_train, y_feat_train)
                        acc = accuracy_score(y_feat_val, model.predict(X_feat_val))
                        results_feat = results_feat.append({'max_depth': j, 'min_samples_split': k, 'min_samples_leaf': l, 'min_weight_fraction_leaf': m, 'max_features': n, 'criterion': i, 'accuracy': acc }, ignore_index=True)

print(results_clean.sort_values(by = 'accuracy', ascending = False).head())
print(results_feat.sort_values(by = 'accuracy', ascending = False).head())

    criterion max_depth min_samples_split min_samples_leaf  \
209      gini       100                11                3   
211      gini       100                11                3   
120      gini       100                 2                2   
48       gini        90                 5                3   
4        gini        90                 2                1   

     min_weight_fraction_leaf max_features  accuracy  
209                     0.000         log2  0.665419  
211                     0.001         sqrt  0.665332  
120                     0.001         auto  0.665043  
48                      0.001         auto  0.665028  
4                       0.001         sqrt  0.664999  
    criterion max_depth min_samples_split min_samples_leaf  \
364   entropy        90                 5                2   
607   entropy       110                 9                2   
66       gini        90                 9                2   
283      gini       110                 9        

In [21]:
results_clean = results_clean.rename(columns={"max_depth": "n_estimators"})
results_feat = results_feat.rename(columns={"max_depth": "n_estimators"})

In [22]:
results_clean.sort_values(by = 'accuracy', ascending = False).head()

,criterion,n_estimators,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_features,accuracy
209,gini,100,11,3,0.000,log2,0.665419
211,gini,100,11,3,0.001,sqrt,0.665332
120,gini,100,2,2,0.001,auto,0.665043
48,gini,90,5,3,0.001,auto,0.665028
4,gini,90,2,1,0.001,sqrt,0.664999


In [23]:
results_feat.sort_values(by = 'accuracy', ascending = False).head()

,criterion,n_estimators,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_features,accuracy
364,entropy,90,5,2,0.001,sqrt,0.665014
607,entropy,110,9,2,0.001,sqrt,0.664739
66,gini,90,9,2,0.001,auto,0.664624
283,gini,110,9,2,0.001,sqrt,0.664566
310,gini,110,11,2,0.001,sqrt,0.664421


In [17]:
test_results = pd.DataFrame(columns=["model", "accuracy"])

y_clean_col = clean_data.state
X_clean_train = dum_clean_data.drop(columns=["state"])
X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X_clean_train, y_clean_col, test_size = 0.3, random_state=1)

y_feat_col = feat_data.state
X_feat_train = dum_clean_data.drop(columns=["state"])
X_feat_train, X_feat_test, y_feat_train, y_feat_test = train_test_split(X_feat_train, y_feat_col, test_size = 0.3, random_state=1)

model1 = RandomForestClassifier(criterion = "gini", n_estimators = 100, min_samples_split = 11, min_samples_leaf = 3, min_weight_fraction_leaf = 0.0, max_features = "log2")
model1.fit(X_clean_train, y_clean_train)
acc = accuracy_score(y_clean_test, model1.predict(X_clean_test))
test_results = test_results.append({"model": "model1", "accuracy": acc}, ignore_index=True)

model2 = RandomForestClassifier(criterion = "gini", n_estimators = 100, min_samples_split = 11, min_samples_leaf = 3, min_weight_fraction_leaf = 0.001, max_features = "sqrt")
model2.fit(X_clean_train, y_clean_train)
acc = accuracy_score(y_clean_test, model2.predict(X_clean_test))
test_results = test_results.append({"model": "model2", "accuracy": acc}, ignore_index=True)

model3 = RandomForestClassifier(criterion = "gini", n_estimators = 100, min_samples_split = 2, min_samples_leaf = 2, min_weight_fraction_leaf = 0.001, max_features = "auto")
model3.fit(X_clean_train, y_clean_train)
acc = accuracy_score(y_clean_test, model3.predict(X_clean_test))
test_results = test_results.append({"model": "model3", "accuracy": acc}, ignore_index=True)

model4 = RandomForestClassifier(criterion = "entropy", n_estimators = 90, min_samples_split = 5, min_samples_leaf = 2, min_weight_fraction_leaf = 0.001, max_features = "sqrt")
model4.fit(X_feat_train, y_feat_train)
acc = accuracy_score(y_feat_test, model4.predict(X_feat_test))
test_results = test_results.append({"model": "model4", "accuracy": acc}, ignore_index=True)

model5 = RandomForestClassifier(criterion = "entropy", n_estimators = 110, min_samples_split = 9, min_samples_leaf = 2, min_weight_fraction_leaf = 0.001, max_features = "sqrt")
model5.fit(X_feat_train, y_feat_train)
acc = accuracy_score(y_feat_test, model5.predict(X_feat_test))
test_results = test_results.append({"model": "model5", "accuracy": acc}, ignore_index=True)

model6 = RandomForestClassifier(criterion = "gini", n_estimators = 90, min_samples_split = 9, min_samples_leaf = 2, min_weight_fraction_leaf = 0.001, max_features = "auto")
model6.fit(X_feat_train, y_feat_train)
acc = accuracy_score(y_feat_test, model6.predict(X_feat_test))
test_results = test_results.append({"model": "model6", "accuracy": acc}, ignore_index=True)

In [18]:
test_results.sort_values(by = 'accuracy', ascending = False)

,model,accuracy
0,model1,0.665820
1,model2,0.664869
5,model6,0.664393
2,model3,0.664029
4,model5,0.664019
3,model4,0.662734


In [25]:
importances = dict()
numbers = model1.feature_importances_
columns = X_clean_train.columns

for i in range(len(numbers)):
    importances[columns[i]] = numbers[i]

dict(sorted(importances.items(), key=lambda item: item[1], reverse=True))

{'usd_goal_real': 0.5378171525143122,
 'category_Tabletop Games': 0.03554841069336905,
 'main_category_Theater': 0.021001095372640975,
 'main_category_Music': 0.018699070360001617,
 'category_Shorts': 0.016977548116307185,
 'main_category_Technology': 0.016520718357250455,
 'category_Apps': 0.016265519759432287,
 'category_Indie Rock': 0.013916229739292548,
 'category_Web': 0.012624754787739098,
 'category_Hip-Hop': 0.012586866857307328,
 'main_category_Food': 0.01197527639557131,
 'main_category_Comics': 0.011694341939367628,
 'main_category_Fashion': 0.0103695191342408,
 'category_Theater': 0.010365046384532778,
 'category_Video Games': 0.009946749284387902,
 'category_Country & Folk': 0.00934788070379927,
 'main_category_Games': 0.007945840089629041,
 'main_category_Crafts': 0.006598509799727212,
 'category_Apparel': 0.006519096305440389,
 'main_category_Dance': 0.006480862949481779,
 'category_Mobile Games': 0.006126666573170681,
 'category_Fiction': 0.005865281093837893,
 'categor